In [ ]:
# Standard dependencies
import os, glob
import json
import pandas as pd
import xml.etree.ElementTree as ET

# OQ dependencies
from openquake.hazardlib import gsim

## User inputs

In [ ]:
# Event name
event = '20150425_M7.8_Gorkha'
trt = "Subduction Interface"

# Folders and paths
json_path = glob.glob(os.path.join('..', '*', event, 'OpenQuake_gmfs',
                                   'shakemap_info.json'))[0]
country = json_path.split(os.sep)[1]
folder_out = os.path.join("..", country, event, "OpenQuake_gmfs")
out_path = os.path.join(folder_out, "gmpe_logic_tree_USGS.xml")


In [ ]:
with open(json_path) as f:
    info = json.load(f)

# Preview
info

## Read supplemental info from USGS

In [ ]:
# Choose imt to calibrate based upon
imt = 'PGA'

# Extract values
gmpes = info['multigmpe'][imt]['gmpes']

# Determine tectonic region types
trt_names = [trt['name'] for trt in gmpes]
trt_weights = info['multigmpe'][imt]['weights']
n_trt = len(trt_names)
print(f"Found {n_trt} tectonic region types: {trt_names}")

# Restructure dictionary
gmmLT = dict.fromkeys(trt_names)
for i in range(n_trt):
    trt = trt_names[i]
    gmmLT[trt] = {
        'weight': trt_weights[i],
        'gmpes': {name.replace('[','').replace(']',''): weight for name, weight in zip(gmpes[i]['gmpes'], gmpes[i]['weights'])}
    }

# Preview
gmmLT

In [ ]:
# Flatten gmmLT

# Create dataframe for each trt
dfs = [None for _ in range(n_trt)]
for i in range(n_trt):
    trt, wt = trt_names[i], trt_weights[i]
    dfs[i] = pd.DataFrame({
                            'gmpe': list(gmmLT[trt]['gmpes'].keys()),
                            'gmpe_weight': list(gmmLT[trt]['gmpes'].values())
                          })
    dfs[i]['trt'] = trt
    dfs[i]['trt_weight'] = wt
    dfs[i]['branch_weight'] = dfs[i]['gmpe_weight'].mul(wt)
    
# Concatenate
df = pd.concat(dfs, axis=0).reset_index().drop(columns='index')
df['branch'] = [f"b{i}" for i in df.index]

# Preview
df

In [ ]:
# Get all available OQ GMPEs
oq_gmpes = [name for name in dir(gsim) if not name.startswith("_")]

# Required GMPEs to be mapped
usgs_gmpes = df.gmpe.unique()

# Print guidance to user
print(f"You need to map OQ GMPE names to the following USGS GMPE names:",
      usgs_gmpes, sep='\n')
# print(*oq_gmpes, sep='\n')

In [ ]:
# Write XML file line by line

# Begin to write file
nrml_file = open(out_path, 'w')

# Header
header_lines = [
    "<?xml version='1.0' encoding='utf-8'?>\n"
    '<nrml xmlns:gml="http://www.opengis.net/gml" xmlns="http://openquake.org/xmlns/nrml/0.4">\n'
    "<logicTree logicTreeID='lt1'>\n"
    f'    <logicTreeBranchSet uncertaintyType="gmpeModel" branchSetID="bs1" applyToTectonicRegionType="{trt}">'
]
for line in header_lines:
    nrml_file.write(f"{line}\n")
    
# Branches
for i, row in df.iterrows():
    branch, gmpe, weight = row.branch, row.gmpe, row.branch_weight
    lines = [
            f"        <logicTreeBranch branchID='{branch}'>",
             "            <uncertaintyModel>",
            f"                {gmpe}",
             '            </uncertaintyModel>',
            f"            <uncertaintyWeight>{weight}</uncertaintyWeight>",
             '        </logicTreeBranch>'
             "\n"
            ]
    for line in lines:
        nrml_file.write(f"{line}\n")
    
# Footer
footer_lines = [
    '    </logicTreeBranchSet>\n'
    '</logicTree>\n'
    "</nrml>"
]
for line in footer_lines:
    nrml_file.write(f"{line}\n")
nrml_file.close()

# Print something
print(f"Wrote to: {out_path}")

In [ ]:
# Ensure that xml is valid
error_msg = f"Invalid XML file: {out_path}"
try:
    ET.parse(out_path)
except ET.ParseError as e:
    error_msg = error_msg + f"\n{e}"
    print(error_msg)